In [1]:
# TEMP: Import lagom
# Not useful once lagom is installed
import sys
sys.path.append('/home/zuo/Code/lagom/')

### use functiontools.partial to set make_env function without argument but internally

In [18]:
import numpy as np

np.swapaxes()

<module 'numpy' from '/home/zuo/anaconda3/envs/RL_server/lib/python3.6/site-packages/numpy/__init__.py'>

# VecEnv

In [ ]:
import numpy as np
from multiprocessing import Process  # easier code than threading
from multiprocessing import Pipe  # Much faster than Queue


class CloudpickleWrapper(object):
    """
    Uses cloudpickle to serialize contents (otherwise multiprocessing tries to use pickle)
    """
    def __init__(self, x):
        self.x = x
        
    def __call__(self):
        return self.x()
    
    def __getstate__(self):
        import cloudpickle
        return cloudpickle.dumps(self.x)
    def __setstate__(self, ob):
        import pickle
        self.x = pickle.loads(ob)


def worker(child_conn, parent_conn, make_env):
    parent_conn.close()
    # Create an environment
    env = make_env()
    
    while True:
        cmd, data = child_conn.recv()
        if cmd == 'step':
            obs, reward, done, info = env.step(data)
            if done:
                obs = env.reset()  # TODO: why reset
            child_conn.send([obs, reward, done, info])
        elif cmd == 'reset':
            obs = env.reset()
            child_conn.send(obs)
        elif cmd == 'reset_task':
            obs = env.reset_task()
            child_conn.send(obs)
        elif cmd == 'close':
            child_conn.close()
            break
        elif cmd == 'get_spaces':
            child_conn.send([env.observation_space, env.action_space])
        else:
            raise NotImplementedError

class SubprocVecEnv(VecEnv):
    """
    Run a list of environment in subprocesses
    """
    def __init__(self, list_make_env):
        self.waiting = False
        self.closed = False
        self.num_envs = len(list_make_env)
        self.parent_conns, self.child_conns = zip(*[Pipe() for _ in range(self.num_envs)])
        self.processes = []
        for parent_conn, child_conn, make_env in zip(self.parent_conns, self.child_conns, list_make_env):
            self.processes.append(Process(target=worker, args=[child_conn, parent_conn, CloudpickleWrapper(make_env)]))  # TODO: CloudpickleWrapper make_env
        for process in self.processes:
            process.daemon = True  # if the main process crashes, we should not cause things to hang
            process.start()
        for conn in self.child_conns:
            conn.close()
        
        # Obtain observation and action spaces
        self.parent_conns[0].send(['get_spaces', None])
        observation_space, action_space = self.parent_conns[0].recv()
        super().__init__(self.num_envs, observation_space, action_space)
        
    def step_asyn(self, actions):
        for parent_conn, action in zip(self.parent_conns, actions):
            parent_conn.send(['step', action])
            
        self.waiting = True
        
    def step_wait(self):
        observations, rewards, dones, infos = zip(*[parent_conn.recv() for parent_conn in self.parent_conns])
        self.waiting = False
        return np.stack(observations), np.stack(rewards), np.stack(dones), infos
    
    def reset(self):
        for parent_conn in self.parent_conns:
            parent_conn.send(['reset', None])
        return np.stack([parent_conn.recv() for parent_conn in self.parent_conns])
    
    def reset_task(self):
        for parent_conn in self.parent_conns:
            parent_conn.send(['reset_task', None])
        return np.stack([parent_conn.recv() for parent_conn in self.parent_conns])
    
    def close(self):
        if self.closed:
            return
        if self.waiting:
            for parent_conn in self.parent_conns:
                parent_conn.recv()
        for parent_conn in self.parent_conns:
            parent_conn.send(['close', None])
        for process in self.processes:
            process.join()
        self.closed = True

# LSTM network

In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim


import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt


np.random.seed(2)

T = 20
L = 1000
N = 100

x = np.empty((N, L), 'int64')
x[:] = np.array(range(L)) + np.random.randint(-4 * T, 4 * T, N).reshape(N, 1)
data = np.sin(x/T).astype('float32')


class Sequence(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.lstm1 = nn.LSTMCell(1, 51)
        self.lstm2 = nn.LSTMCell(51, 51)
        self.linear = nn.Linear(51, 1)
        
    def forward(self, x, future=0):
        outputs = []
        batch_size, x_size = x.size()
        
        h_t = torch.zeros(batch_size, 51)
        c_t = torch.zeros(batch_size, 51)
        h_t_2 = torch.zeros(batch_size, 51)
        c_t_2 = torch.zeros(batch_size, 51)
        
        chunks = x.chunk(x_size, dim=1)
        
        for x_t in chunks:
            h_t, c_t = self.lstm1(x_t, (h_t, c_t))
            h_t_2, c_t_2 = self.lstm2(h_t, (h_t_2, c_t_2))
            
            output = self.linear(h_t_2)
            
            outputs.append(output)
            
        for _ in range(future):  # if we should predict the future
            h_t, c_t = self.lstm1(output, (h_t, c_t))
            h_t_2, c_t_2 = self.lstm2(h_t, (h_t_2, c_t_2))
            
            output = self.linear(h_t_2)
            
            outputs.append(output)
            
        outputs = torch.stack(outputs, dim=1).squeeze(2)
        
        return outputs
        
if __name__ == '__main__':

    np.random.seed(0)
    torch.manual_seed(0)

    input = torch.from_numpy(data[3:, :-1])
    target = torch.from_numpy(data[3:, 1:])
    test_input = torch.from_numpy(data[:3, :-1])
    test_target = torch.from_numpy(data[:3, 1:])

    seq = Sequence()

    criterion = nn.MSELoss()

    # Use LBFGS since we load whole data to train
    optimizer = optim.LBFGS(seq.parameters(), lr=0.8)

    for i in range(2):
        print(f'STEP: {i}')
        def closure():
            optimizer.zero_grad()
            out = seq(input)
            loss = criterion(out, target)
            print(f'Loss: {loss.item()}')
            loss.backward()
            return loss
        optimizer.step(closure)

        # Prediction
        future = 1000
        pred = seq(test_input, future=future)
        loss = criterion(pred[:, :-future], test_target)
        print(f'Test loss: {loss.item()}')
        y = pred.data.numpy()

        # Drawing
        plt.figure(figsize=(30, 10))
        plt.xlabel('x')
        plt.ylabel('y')
        plt.xticks(fontsize=20)
        plt.yticks(fontsize=20)
        def draw(yi, color):
            plt.plot(np.arange(input.size(1)), yi[:input.size(1)], color, linewidth=2.0)
            plt.plot(np.arange(input.size(1), input.size(1)+future), yi[input.size(1):], color + ':', linewidth=2.0)
        draw(y[0], 'r')
        draw(y[1], 'g')
        draw(y[2], 'b')
        
        plt.savefig(f'logs/{i}.pdf')

# VAE

In [20]:
# TEMP: Import lagom
# Not useful once lagom is installed
import sys
sys.path.append('/home/zuo/Code/lagom/')

torch.Size([2, 10])


In [ ]:
list(train_loader)

In [32]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
from torchvision.utils import save_image



# Set random seed
torch.manual_seed(config['seed'])

# Automatic check if there is GPU
cuda = torch.cuda.is_available()

# Define device
config['device'] = torch.device('cuda' if cuda else 'cpu')
device = config['device']

# Define GPU-dependent keywords for DataLoader
if cuda:
    kwargs = {'num_workers': 1, 'pin_memory': True}
else:
    kwargs = {}
    
# Create dataset for training and testing
train_dataset = datasets.MNIST('data/', 
                               train=True, 
                               download=True, 
                               transform=transforms.ToTensor())
test_dataset = datasets.MNIST('data/', 
                              train=False, 
                              transform=transforms.ToTensor())
    
train_loader = DataLoader(train_dataset, 
                          batch_size=config['batch_size'], 
                          shuffle=True, 
                          **kwargs)
test_loader = DataLoader(test_dataset, 
                         batch_size=config['batch_size'], 
                         shuffle=True, 
                         **kwargs)



from lagom.core.networks import VAE

model = VAE(input_dim=784, 
            encoder_sizes=[400], 
            encoder_nonlinearity=F.relu, 
            latent_dim=20, 
            decoder_sizes=[400], 
            decoder_nonlinearity=F.relu)
model = model.to(config['device'])

optimizer = optim.Adam(model.parameters(), lr=1e-3)





                
       
from lagom.core.utils import Logger
logger = Logger(name='logger')
engine = Engine(model, optimizer, train_loader, test_loader, config, logger)
            
        
        

for epoch in range(1, config['epochs'] + 1):
    engine.train()
    engine.eval()
    
    """
    with torch.no_grad():
        sample = torch.randn(64, 20).to(device)
        sample = model.decode(sample).cpu()
        save_image(sample.view(64, 1, 28, 28),
                   'data/sample_' + str(epoch) + '.png')"""

Training iteration #1: 546.5628051757812
Training iteration #100: 177.91822814941406
Training iteration #200: 151.71519470214844
Training iteration #300: 138.11561584472656
Training iteration #400: 135.24795532226562
====> Average loss: 165.32561080729167
====> Test set loss: 119.13991490478516


In [ ]:
config = {}
config['batch_size'] = 128
config['epochs'] = 1
config['seed'] = 1
config['log_interval'] = 100

In [ ]:
====> Epoch: 1 Average loss: 164.4686
====> Test set loss: 119.2985

In [ ]:
165.3256